In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from scipy.spatial import ConvexHull
from sklearn.preprocessing import StandardScaler

In [ ]:
# Calculate z-scores for x and y columns
# make this code into a function that takes a dataframe and returns the cleaned dataframe

from scipy.stats import zscore
def clean_outliers(df):
    z_scores_x = zscore(df.iloc[:,0])
    z_scores_y = zscore(df.iloc[:,1])

    # Define threshold for z-score (e.g., 3)
    threshold = 3

    # Identify outliers based on z-scores
    outliers_x = df[abs(z_scores_x) > threshold]
    outliers_y = df[abs(z_scores_y) > threshold]

    # Remove outliers from DataFrame
    cleaned_df = df[~df.index.isin(outliers_x.index) & ~df.index.isin(outliers_y.index)]
    return cleaned_df

In [ ]:
from moviepy.editor import VideoFileClip
video_folder = '/Users/andrei-macpro/Documents/Data/videos/play_videos' 

durations = []
file_names = []

for folder_name in sorted(os.listdir(video_folder)):
    if folder_name == ".DS_Store":
        continue
    file_path = os.path.join(f_name, folder_name)
    if os.path.isfile(file_path):
        clip = VideoFileClip(file_path)
        durations.append(clip.duration)
        file_names.append(folder_name.split('.')[0])

durations = pd.DataFrame({'file_name': file_names, 'duration': durations})

In [ ]:
tracking_folder = '/Users/andrei-macpro/Documents/Data/openpose/play/tracking/tracking' 

In [ ]:
# now do it for all videos
stats = {}
scaler = StandardScaler()
for folder_name in sorted(os.listdir(tracking_folder)):
    if folder_name == ".DS_Store":
        continue
    file_path = os.path.join(tracking_folder, folder_name)
    tracks = {}
    for file in sorted(os.listdir(file_path)):
                # load the file in a pandas dataframe
            if file == ".DS_Store":
                continue    
            filepath = os.path.join(tracking_folder,folder_name, file)
            df = pd.read_csv(filepath, index_col=0)
            df = df[~df.index.duplicated(keep='first')]
            df.columns = ['x_' + file.split('.')[0], 'y_' + file.split('.')[0]]
            df = clean_outliers(df)
            tracks[file.split('.')[0]] = df


    combined_data = pd.concat([df for df in tracks.values()], axis=1)
    combined_scaled = scaler.fit_transform(combined_data)
    # Convert the scaled data back to a DataFrame
    combined_scaled_df = pd.DataFrame(combined_scaled, index=combined_data.index, columns=combined_data.columns)

# Split the combined_scaled_df back into separate DataFrames and assign them back to the tracks dictionary
    for track_name in tracks.keys():
        x_cols = [col for col in combined_scaled_df.columns if col.startswith('x_' + track_name)]
        y_cols = [col for col in combined_scaled_df.columns if col.startswith('y_' + track_name)]
        tracks[track_name] = combined_scaled_df[x_cols + y_cols]

    # Calculate  hull areas
    for track_name, track_df in tracks.items():
        duration = durations.loc[durations['file_name'] == folder_name]['duration'].values[0]
        frame_indices = track_df.index
        points = track_df[['x_' + track_name, 'y_' + track_name]].values
        points = points[~np.isnan(points).any(axis=1)]
         # Calculate the Convex Hull of the points
        hull = ConvexHull(points)
        hull_area = hull.area
 
        
        
        if folder_name not in stats:
            stats[folder_name] = {}
        stats[folder_name][track_name] = {'area_travelled':hull_area/duration*100}
 
# normalize the distances (mean, variance etc) by the duration of the video
    

In [ ]:
# Convert the stats dictionary to a DataFrame
df = pd.DataFrame.from_dict({(i,j): stats[i][j] 
                           for i in stats.keys() 
                           for j in stats[i].keys()},
                           orient='index')

# Set the index names
df.index.names = ['s_id', 'track_name']

In [ ]:
# Reset only the second level of the index (track_name)
df.reset_index(level=1, inplace=True)

# Create new DataFrame to hold results
new_df = pd.DataFrame(index=df.index.unique())

# Loop over each unique track_name
for track_name in df['track_name'].unique():
    # Select rows for this track_name
    temp_df = df[df['track_name'] == track_name].copy()
    # Drop the 'track_name' column as it's no longer needed
    temp_df.drop(columns=['track_name'], inplace=True)
    # Add the track_name as a prefix to each column name
    temp_df.columns = [f'{track_name}_{col}' for col in temp_df.columns]
    # Add the results to new_df
    new_df = new_df.join(temp_df)

new_df

In [ ]:
# save df to /Users/andrei-macpro/Documents/Data/tracking/features
new_df.to_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/area_travelled_each.csv')